In [ ]:
%pylab ipympl
from nugridpy import mesa as ms
from nugridpy import nugridse as mp

# default data location (try mirror if default
# is not available)
import matplotlib.pyplot as plt

### The MESA stellar evolution model

By default MESA is putting out two types of data. History data provides the time evolution of scalar quantities, one per time step. This data can be accessed with the `mesa.star_log` (or `mesa.history_data` which is the same) class.

MESA also outputs profile data at select time steps. Profiles are available via the `mesa_profile` class.

#### History data
Initialise the 2 solar-mass Z=0.02 MESA stellar evolution model from set1.2 using the seeker method:

In [ ]:
# is not available)
data_dir="/data/ASDR/NuGrid"  

# mirror NuGrid data location - uncomment both of the following lines
# ![ ! -h /user/data ] && ln -s /data/nugrid_data /user/data
# data_dir = '/user'

ms.set_nugrid_path(data_dir)
mp.set_nugrid_path(data_dir)

In [ ]:
s=ms.star_log(mass=5, Z=0.001)

Let's now see what the model looks like by making some Kippenhahn diagrams

In [ ]:
m_rem = s.get("h1_boundary_mass")[-1]
m_rem

In [ ]:
s.get("star_mass")[-1]

In [ ]:
modelmax = int(s.get("model_number")[-1])
modelmax

In [ ]:
ifig=111;close(ifig);figure(ifig)
s.kippenhahn_CO(111,'model')

What's happening at the centre of the star? ...

In [ ]:
ifig=101;close(ifig);figure(ifig)
s.tcrhoc()
axis([0,7,7,8.5])

... and at the surface?

In [ ]:
ifig=102;close(ifig);figure(ifig)
s.hrd_new()
legend(loc='lower right').draw_frame(False)

In [ ]:
ifig=107;close(ifig);figure(ifig)
s.kip_cont(ifig=ifig, boundaries=True,engenlevels=[1e2,1e6,1e12])

Now that we have an overview of the stellar evolution of this $2M_\odot$ star let's have a closer look at the nucleosynthesis.

### The NuGrid `mppnp` post-processing data
The NuGrid `mppnp` code would post-process detailed MESA hdf5 output. This post-processing data is available via the NuGridPy `nugridse` class. 

Initialise the 2 solar-mass Z=0.02 NuGrid nucleosynthesis data from set1.2 using the seeker method:

In [ ]:
pt = mp.se(sedir=f"/data/nugrid_data/set1upd/{setname}/ppd_wind/RUN_set1upd_{modelname}/H5_out/")

#### What quantities are available in this data type, and what are the units?
Each of the _se_ file sets (in fact each of the dozens of hdf5 files that make up the data set for one mass/metallicty combination, or stellar evolution track) has three types of data contained in them:

data type access | content 
----------------|---------
 `pt.se.hattrs` |  a header section that holds the _header attributes_, including units in the form of factors so that if applied with the quantities the result is in cgs units 
`pt.se.cattrs` | for each cycle (or time step) the _cycle attributes_ are a number of scalar global quantities, such as total mass or star age
`pt.se.dcols` | again, for each time step these are the vector quantities available, i.e. the data table columns; one of the data columns, _iso_massf_ is in fact a matrix where the matrix columns are different species, i.e. a radial vector of species vectors

In [ ]:
pt.se.hattrs

In [ ]:
pt.se.cattrs

In [ ]:
pt.se.dcols

We can use the `get` method to pull any of the data in

#### Example
What is the unit of the temperature?

In [ ]:
pt.get('temperature_unit')/1e9

In [ ]:
plt.figure()
pt.plot('mass','temperature',fname=1000, shape=':')
plt.ylabel('$T / 10^9 \mathrm{K}$')
plt.xlabel('$m_r/M_\odot$');

#### Abundance profiles 

##### A number of isotopes in the H shell 
In the stellar evolution analysis we found that a H-burning shell inbetween thermal pulses could be found at cycle 18000. Let's plot a few isotopes that are being created or destroyed in this H-burning shell:

In [ ]:
# pt.abu_profile?

and the nuclides that are being created or destroyed in this part of the star:

In [ ]:
species=['H-1','He-4','C-12','C-13','N-14','O-16']
pt.abu_profile(isos=species, fname=18000, logy=True)


##### Abundance profile at the time of formation of the partial-mix zone

In [ ]:
species=['H-1','C-12','C-13','N-14','Fe-56','Sr-86','Ba-138','Pb-206']

In [ ]:
Xs = pt.get_elemental_abunds(modelmax)

In [ ]:
pt.se.dcols

In [ ]:
len(pt.se.isotopes)

In [ ]:
EM_wind = calc_wind_yields(pt)

In [ ]:
calc_yield_wind("H-1", end=100)

In [ ]:
rem_yield = calc_rem_yield(pt)

In [ ]:
def calc_rem_yield(pt, modelmax=modelmax, m_rem=m_rem):
    mass = pt.get(modelmax, "mass")
    filt = mass > m_rem
    
    Xs = pt.get(modelmax, "iso_massf")
    
    dm = np.gradient(mass)[filt]
    
    ele_mass = Xs * dm

    
    return np.sum(ele_mass, dims=1)

In [ ]:
calc_yield("C-12")

In [ ]:
len(pt.get(modelmax,'iso_massf',"H-1"))

In [ ]:
len(Xs[1])

In [ ]:
len(Xs)

In [ ]:
pt.abu_profile(isos=species, fname=45500, logy=True, colourblind=True)
ylim(-9,0)
xlim(0.603,0.6033)
title("Formation of the $^\mathsf{13}\mathsf{C}$ pocket: the partial H-$^\mathsf{12}\mathsf{C}$ zone")

In [ ]:
ifig=124;close(ifig);figure(ifig)
pt.abu_profile(isos=species, ifig=ifig, fname=modelmax, logy=True, colourblind=False)
ylim(-9,0)
title("Final s-process formed in $^\mathsf{13}\mathsf{C}$ pocket: end of intershell period")

In [ ]:
ifig=124;close(ifig);figure(ifig)
pt.abu_profile(isos=species, ifig=ifig, fname=47222, logy=True, colourblind=False)
ylim(-9,0)
xlim(0.60312,0.6032)
title("Final s-process formed in $^\mathsf{13}\mathsf{C}$ pocket: end of intershell period")